Script for transforming csv file into jsonl for training a spacy model. 
This is the most straightforward setup, where every abstract has an 'accept' annotation for its
corresponding journal, and a 'reject' annoation for every other journal.

Including rejected annotations is necessary. But will lead to lower absolute confidence scores for abstracts which could belong in more than one journal. Better labelling could be done manually, closely matching journals culd be clustered into new classifiers, or develop a user-based metric for interpreting statistical weights. It may well be possible to build a custom recipe to assign ('reject')labels mid-trianing - which would significantly decrease memory demand. 

https://support.prodi.gy/t/best-practices-for-text-classifier-annotations/135

In [2]:
import pandas as pd
from tqdm import tqdm
import json

In [4]:
#csv with 'abstract' column and 'journal code' column
df = pd.read_csv('/path/to/abstract.csv')
journal_list = df['journal code'].unique().tolist()

In [2]:
generate list of json accept annotations for every abstract
accept_list = []
for i in tqdm(range(0,df.shape[0])):
    accept_list.append({"answer":"accept","text":r'"'+df['abstract'][i]+'"',"label":df['journal code'][i]}) 

In [6]:
def jsonlReject(text):
    return {"answer":"reject","text":r'"'+text+'"',"label":code}

In [3]:
generate list of json reject annotations for every abstract - 
with every abstract rejected by every journal to which it does not correspond
reject_list = []
for code in tqdm(journal_list):
    rej_code_bool = df['journal code'] != code
    rej_df = df[rej_code_bool]
    reject_list.extend(rej_df['abstract'].apply(lambda text:jsonlReject(text)))

In [34]:
jsonl = accept_list
jsonl.extend(reject_list)

In [29]:
with open('/path/to/abstract.jsonl', 'w') as outfile:
    for entry in jsonl:
        json.dump(entry, outfile)
        outfile.write('\n')